In [1]:
import numpy as np
import cv2
import math

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
background = None
accumulated_weight = 0.5

roi_top = 20
roi_bottom = 400
roi_right = 300
roi_left = 600

In [4]:
def calc_accum_avg(frame,accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame,background,accumulated_weight)

In [5]:
def segment(frame,threshold=25):
    
    global background
    
    diff = cv2.absdiff(background.astype("uint8"),frame)
    _,thresholded = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    contours,hierarchy =  cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours,key = cv2.contourArea)
        return (thresholded,hand_segment)

In [6]:
def dist(a,b):
    return math.sqrt((a[0] - b[0])**2 + (b[1] - a[1])**2)

In [7]:
def filter_points(points,filterValue):
    for i in range(len(points)):
        for j in range(i+1,len(points)):
            if points[i] and points[j] and dist(points[i] , points[j]) < filterValue:
                points[j] = None
    filtered = []
    for point in points:
        if point is not None:
            filtered.append(point)
    return filtered

In [8]:
def plot(frame,points):
    for point in points:
        cv2.circle(frame,point,radius = 5,color = (0,0,255),thickness = -1)

In [12]:
cam = cv2.VideoCapture(0)
#cam.set(cv2.CAP_PROP_FPS,90)
num_frames = 0
screen = np.zeros((380,300))

curr = None
prev = None

# keep looping, until interrupted
while True:

    ret, frame = cam.read()

    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    #print(roi.shape)
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:

        hand = segment(gray)

        if hand is not None:
            thresholded, hand_segment = hand
            masked = cv2.bitwise_and(roi,roi,mask = thresholded)
            #cv2.imshow("Masked",masked)
            #cv2.imshow("Thesholded", thresholded)
            contours,hierarchy = cv2.findContours(thresholded,
                                                  cv2.RETR_TREE,
                                                  cv2.CHAIN_APPROX_SIMPLE)
            palm_area = 0
            flag = None
            cnt = None
            
            for (i,c) in enumerate(contours):
                area = cv2.contourArea(c)
                if area>palm_area:
                    palm_area = area
                    flag = i
            if flag is not None and palm_area > 10000:
                cnt = contours[flag]
                cv2.drawContours(masked,[cnt],0,(0,255,0),2)
                cv2.imshow("roi_contours",masked)
                points = []
                hull = cv2.convexHull(cnt,returnPoints = False)
                defects = cv2.convexityDefects(cnt,hull)
                
                for i in range(defects.shape[0]):
                    s,e,f,d, = defects[i,0]
                    end = tuple(cnt[e][0])
                    points.append(end)
    
                filtered = filter_points(points,50)#.sort(key = lambda x:x[1])
                filtered.sort(key = lambda point : point[1])
                fingers = [pt for (idx,pt) in zip(range(1),filtered)]
                plot(masked,fingers)
    
                cv2.imshow("roi_contours",masked)
               
                prev = curr
                curr = fingers[0]
                
                if prev and curr:
                    cv2.line(screen,prev,curr,(255,255,255),2)
                cv2.imshow("Drawing",screen)
                
                k = cv2.waitKey(10)
                if k == ord('a'):
                    screen = get_new_screen()
                      
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    num_frames += 1
    cv2.imshow("Finger Count", frame_copy)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()

#img = process_img(screen)